# Installing required libraries

In [1]:
!conda install polars -y -q

Channels:
 - conda-forge
 - nvidia
 - pytorch
Platform: linux-64
Solving environment: ...working... done

# All requested packages already installed.



# Importing required libraries

In [2]:
import polars as pl
import numpy as np
import matplotlib.pyplot as plt

# Config file (Paths of datasets to be joined)

In [97]:
class config:
    target_path = './Determine_modeling_datasets/Determine_modeling_age_group_outcome_data.parquet'
    demo_path = './Determine_modeling_datasets/Determine_modeling_demographics_data.parquet'
    
    med_path = './Determine_modeling_datasets/Determine_modeling_medications_thrs_50.parquet'
    diag_path = './Determine_modeling_datasets/Determine_modeling_diagnoses_phemap_thrs_50_without_icd10z.parquet'
    lab_path = './Determine_modeling_datasets/Determine_modeling_labresults_median_domain_expert_thrs_50.parquet'
    
    bmi_path = './Determine_cohort_files/Determine_modeling_bmi_data_manual.parquet'
    dia_bp_path = './Determine_cohort_files/Determine_modeling_diastolic_bp_data.parquet'
    sys_bp_path = './Determine_cohort_files/Determine_modeling_systolic_bp_data.parquet'
    cvs_path = './Determine_modeling_datasets/Determine_cvs_pivot.parquet'
    
    sdoh_screening_path = './Determine_modeling_datasets/Determine_sdoh_screening_pivot.parquet'
    
    use_bmi_bp = True
    use_cvs = True
    
    sdoh_screening_experiment = True
    

# Joining the datasets

In [98]:
target_df = pl.scan_parquet(config.target_path)
demo_df = pl.scan_parquet(config.demo_path)
med_df = pl.scan_parquet(config.med_path)
diag_df = pl.scan_parquet(config.diag_path)
lab_df = pl.scan_parquet(config.lab_path)

bmi_df = pl.scan_parquet(config.bmi_path)
dia_bp_df = pl.scan_parquet(config.dia_bp_path)
sys_bp_df = pl.scan_parquet(config.sys_bp_path)
cvs_df = pl.scan_parquet(config.cvs_path)
sdoh_screening_df = pl.scan_parquet(config.sdoh_screening_path)

In [99]:
sdoh_screening_df.collect().head()

PATIENT_NUM,SDH:ADV0041 _0.0,SDH:ADV0041 _1.0,SDH:ADV0051 _0.0,SDH:ADV0051 _1.0,SDH:ADV0046 _0.0,SDH:ADV0046 _1.0,SDH:ADV0047 _0.0,SDH:ADV0047 _1.0,SDH:ADV0048 _0.0,SDH:ADV0048 _1.0,SDH:ADV0052 _0.0,SDH:ADV0052 _1.0,SDH:ADV0050 _0.0,SDH:ADV0050 _1.0,SDH:ADV0049 _0.0,SDH:ADV0049 _1.0,SDH:ADV0043 _0.0,SDH:ADV0043 _1.0,SDH:ADV0054 _0.0,SDH:ADV0054 _1.0,SDH:ADV0053 _0.0,SDH:ADV0053 _1.0,SDH:ADV0059 _0.0,SDH:ADV0059 _1.0,SDH:ADV0060 _0.0,SDH:ADV0060 _1.0,SDH:ADV0061 _0.0,SDH:ADV0061 _1.0,SDH:ADV0044 _0.0,SDH:ADV0044 _1.0,SDH:ADV0062 _0.0,SDH:ADV0062 _1.0,SDH:ADV0101 _0.0,SDH:ADV0101 _1.0,SDH:ADV0064 _0.0,SDH:ADV0064 _1.0,…,SDH:ADV0096 _1.0,SDH:ADV0083 _0.0,SDH:ADV0083 _1.0,SDH:ADV0097 _0.0,SDH:ADV0097 _1.0,SDH:ADV0084 _0.0,SDH:ADV0084 _1.0,SDH:ADV0102 _0.0,SDH:ADV0102 _1.0,SDH:ADV0099 _0.0,SDH:ADV0099 _1.0,SDH:ADV0103 _0.0,SDH:ADV0103 _1.0,SDH:ADV0111 _0.0,SDH:ADV0111 _1.0,SDH:ADV0113 _0.0,SDH:ADV0113 _1.0,SDH:ADV0104 _0.0,SDH:ADV0104 _1.0,SDH:ADV0261 _0.0,SDH:ADV0261 _1.0,SDH:ADV0114 _0.0,SDH:ADV0114 _1.0,SDH:ADV0115 _0.0,SDH:ADV0115 _1.0,SDH:ADV0116 _0.0,SDH:ADV0116 _1.0,SDH:ADV0117 _0.0,SDH:ADV0117 _1.0,SDH:ADV0118 _0.0,SDH:ADV0118 _1.0,SDH:ADV0262 _0.0,SDH:ADV0262 _1.0,SDH:ADV0400 _0.0,SDH:ADV0400 _1.0,SDH:ADV0401 _0.0,SDH:ADV0401 _1.0
i64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,…,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8
1766173,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2011646,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
445436,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
5646019,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3500474,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [100]:
'1010.0' in diag_df.collect().columns

False

In [101]:
print("Number of unique patient records in demographics dataframe: ", len(np.unique(demo_df.select('Patient_num')
                                                                                    .collect()['Patient_num'].to_list())))
print("Number of unique patient records in diagnoses dataframe: ", len(np.unique(diag_df.select('PATIENT_NUM')
                                                                                    .collect()['PATIENT_NUM'].to_list())))
print("Number of unique patient records in medications dataframe: ", len(np.unique(med_df.select('PATIENT_NUM')
                                                                                    .collect()['PATIENT_NUM'].to_list())))
print("Number of unique patient records in labresults dataframe: ", len(np.unique(lab_df.select('PATIENT_NUM')
                                                                                    .collect()['PATIENT_NUM'].to_list())))
print("Number of unique patient records in BMI dataframe: ", len(np.unique(bmi_df.select('PATIENT_NUM')
                                                                                    .collect()['PATIENT_NUM'].to_list())))
print("Number of unique patient records in Dia bp dataframe: ", len(np.unique(dia_bp_df.select('PATIENT_NUM')
                                                                                    .collect()['PATIENT_NUM'].to_list())))
print("Number of unique patient records in Sys dataframe: ", len(np.unique(sys_bp_df.select('PATIENT_NUM')
                                                                                    .collect()['PATIENT_NUM'].to_list())))

Number of unique patient records in demographics dataframe:  826658
Number of unique patient records in diagnoses dataframe:  664510
Number of unique patient records in medications dataframe:  531310
Number of unique patient records in labresults dataframe:  406036
Number of unique patient records in BMI dataframe:  657056
Number of unique patient records in Dia bp dataframe:  676081
Number of unique patient records in Sys dataframe:  677752


In [102]:
med_pids = np.unique(med_df.select(['PATIENT_NUM']).collect()['PATIENT_NUM'].to_list())
dx_pids = np.unique(diag_df.select(['PATIENT_NUM']).collect()['PATIENT_NUM'].to_list())
lab_pid = np.unique(lab_df.select(['PATIENT_NUM']).collect()['PATIENT_NUM'].to_list())

if config.sdoh_screening_experiment:
    sdoh_screening_pids = np.unique(sdoh_screening_df.select(['PATIENT_NUM']).collect()['PATIENT_NUM'].to_list())
    result = (set(med_pids) | set(dx_pids) | set(lab_pid)) & set(sdoh_screening_pids)
    result_list = list(result)
    print("Number of patient ids that have data in atleast one of the medications, lab_results, diagnoses and sdoh_screening: ",len(result_list))

else:
    
    # Convert lists to sets and use the union operation
    result = set(med_pids) | set(dx_pids) | set(lab_pid)
    result_list = list(result)
    print("Number of patient ids that have data in atleast one of the medications, lab_results, diagnoses : ",len(result_list))


Number of patient ids that have data in atleast one of the medications, lab_results, diagnoses and sdoh_screening:  43624


In [103]:
target_df_filtered = target_df.filter(pl.col('PATIENT_NUM').is_in(result_list))
demo_df_filtered = demo_df.filter(pl.col('Patient_num').is_in(result_list))

In [104]:
temp_df = target_df_filtered.join(demo_df_filtered, how = 'left', left_on = 'PATIENT_NUM', right_on = 'Patient_num')

In [105]:
temp_df.columns

/tmp/ipykernel_26207/3421553128.py:1: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  temp_df.columns


['PATIENT_NUM',
 'Age_group',
 'FirstOutcomeDate',
 'Outcome',
 'Sex_CD',
 'Race_CD',
 'Hispanic_CD',
 'Gender_CD']

In [106]:
temp_df = temp_df.join(med_df, on = 'PATIENT_NUM', how = 'left')
temp_df = temp_df.fill_null(0)

In [107]:
temp_df.head().collect()

PATIENT_NUM,Age_group,FirstOutcomeDate,Outcome,Sex_CD,Race_CD,Hispanic_CD,Gender_CD,phenazopyridine,nitrofurantoin,lisinopril_hydrochlorothiazide,amlodipine,cephalexin,albuterol,nystatin,metronidazole,mometasone_formoterol,varicella-zoster virus vaccine live (Oka-Merck) strain,potassium chloride,chlorthalidone,ketorolac,azithromycin,triamcinolone,gabapentin,acetaminophen_hydrocodone,capsaicin,naproxen,tramadol,ibuprofen,codeine_promethazine,omeprazole,clotrimazole,lorazepam,fluticasone,calcium carbonate,loratadine,paroxetine,…,candesartan,liraglutide,tretinoin_fluocinolone_hydroquinone,carboxymethylcellulose_glycerin,"L1 protein, human papillomavirus type 31 vaccine_L1 protein, human papillomavirus type 33 vaccine_L1 protein, human papillomavirus type 45 vaccine_L1 protein, human papillomavirus type 52 vaccine_L1 protein, human papillomavirus type 58 vaccine_L1 protein, human papillomavirus type 11 vaccine_L1 protein, human papillomavirus type 16 vaccine_L1 protein, human papillomavirus type 18 vaccine_L1 protein, human papillomavirus type 6 vaccine",neomycin,lamivudine,magnesium sulfate,Neisseria meningitidis serogroup A capsular polysaccharide diphtheria toxoid protein conjugate vaccine_Neisseria meningitidis serogroup C capsular polysaccharide diphtheria toxoid protein conjugate vaccine_Neisseria meningitidis serogroup W-135 capsular polysaccharide diphtheria toxoid protein conjugate vaccine_Neisseria meningitidis serogroup Y capsular polysaccharide diphtheria toxoid protein conjugate vaccine,maraviroc,rilpivirine,calcium acetate,eflornithine,sodium bicarbonate,lindane,entecavir,colestipol,hydrochlorothiazide_irbesartan,aspirin_caffeine_butalbital_codeine,indapamide,risedronate,vancomycin,leuprolide,amlodipine_olmesartan_hydrochlorothiazide,wheat dextrin,epinastine,desipramine,camphor_menthol,rx_cui:993763,"estrogens, esterified (USP)",trospium,methylcellulose,clorazepate,rx_cui:1291624,spinosad,sofosbuvir,ribavirin
i64,str,datetime[μs],i64,str,str,str,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
576517,"""45-54""",null,0,"""F""","""05""","""N""","""W""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5886210,"""55-64""",null,0,"""F""","""UN""","""Y""","""W""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
390161,"""18-34""",null,0,"""F""","""03""","""UN""","""UN""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5378498,"""55-64""",null,0,"""F""","""05""","""UN""","""W""",0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3476481,"""55-64""",null,0,"""F""","""UN""","""UN""","""UN""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [108]:
temp_df = temp_df.join(diag_df, on = 'PATIENT_NUM', how = 'left')
temp_df = temp_df.fill_null(0)

In [109]:
temp_df.head().collect()

PATIENT_NUM,Age_group,FirstOutcomeDate,Outcome,Sex_CD,Race_CD,Hispanic_CD,Gender_CD,phenazopyridine,nitrofurantoin,lisinopril_hydrochlorothiazide,amlodipine,cephalexin,albuterol,nystatin,metronidazole,mometasone_formoterol,varicella-zoster virus vaccine live (Oka-Merck) strain,potassium chloride,chlorthalidone,ketorolac,azithromycin,triamcinolone,gabapentin,acetaminophen_hydrocodone,capsaicin,naproxen,tramadol,ibuprofen,codeine_promethazine,omeprazole,clotrimazole,lorazepam,fluticasone,calcium carbonate,loratadine,paroxetine,…,phe_145.2,phe_426.23,phe_590,phe_349.0,phe_198.5,phe_989.0,phe_375.0,ICD10CM:N89,phe_284.0,phe_686.4,phe_255.12,phe_149.1,ICD10CM:F34,ICD10CM:F17,phe_750.22,phe_151.0,phe_426.3,phe_374.6,phe_615,phe_709.5,phe_281.1,ICD10CM:R10,ICD10CM:R63,ICD10CM:F41,ICD10CM:N93,ICD10CM:R79,ICD10CM:L30,ICD10CM:R94,ICD10CM:N95,ICD10CM:B18,ICD10CM:N94,ICD10CM:R03,ICD10CM:R68,ICD10CM:E87,ICD10CM:R92,ICD10CM:G89,ICD10CM:R09
i64,str,datetime[μs],i64,str,str,str,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
576517,"""45-54""",null,0,"""F""","""05""","""N""","""W""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5886210,"""55-64""",null,0,"""F""","""UN""","""Y""","""W""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
390161,"""18-34""",null,0,"""F""","""03""","""UN""","""UN""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5378498,"""55-64""",null,0,"""F""","""05""","""UN""","""W""",0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3476481,"""55-64""",null,0,"""F""","""UN""","""UN""","""UN""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [110]:
temp_df = temp_df.join(lab_df, on = 'PATIENT_NUM', how = 'left')
temp_df = temp_df.fill_null(-100)

In [111]:
temp_df = temp_df.with_columns([
    pl.col('Sex_CD').str.replace(r'^(NI|OT|UN)$', 'UNK', literal=False).alias('Sex_CD'),
    pl.col('Race_CD').str.replace(r'^(NI|OT|UN)$', 'UNK', literal=False).alias('Race_CD'),
    pl.col('Hispanic_CD').str.replace(r'^(R|NI|UN)$', 'UNK', literal=False).alias('Hispanic_CD'),
    pl.col('Gender_CD').str.replace(r'^(OT|NI|UN)$', 'UNK', literal=False).alias('Gender_CD')
])

In [112]:
if config.use_bmi_bp:
    temp_df = temp_df.join(bmi_df, on = 'PATIENT_NUM', how = 'left')
    temp_df = temp_df.fill_null(-100)
    temp_df = temp_df.join(dia_bp_df, on = 'PATIENT_NUM', how = 'left')
    temp_df = temp_df.fill_null(-100)
    temp_df = temp_df.join(sys_bp_df, on = 'PATIENT_NUM', how = 'left')
    temp_df = temp_df.fill_null(-100)
    

In [113]:
if config.use_cvs:
    temp_df = temp_df.join(cvs_df, on = 'PATIENT_NUM',how ='left')
    temp_df = temp_df.fill_null(-100)

In [114]:
temp_df.head().collect()

PATIENT_NUM,Age_group,FirstOutcomeDate,Outcome,Sex_CD,Race_CD,Hispanic_CD,Gender_CD,phenazopyridine,nitrofurantoin,lisinopril_hydrochlorothiazide,amlodipine,cephalexin,albuterol,nystatin,metronidazole,mometasone_formoterol,varicella-zoster virus vaccine live (Oka-Merck) strain,potassium chloride,chlorthalidone,ketorolac,azithromycin,triamcinolone,gabapentin,acetaminophen_hydrocodone,capsaicin,naproxen,tramadol,ibuprofen,codeine_promethazine,omeprazole,clotrimazole,lorazepam,fluticasone,calcium carbonate,loratadine,paroxetine,…,LOINC:41649-5,LOINC:13801-6,LOINC:49502-8,LOINC:15061-5,LOINC:6874-2,LOINC:32046-5,LOINC:2240-0,LOINC:3054-4,LOINC:6942-7,LOINC:29239-1,LOINC:42245-1,LOINC:1854-9,LOINC:35707-9,LOINC:2148-5,LOINC:38252-3,LOINC:72888-1,LOINC:83102-4,LOINC:38506-2,LOINC:26499-4,LOINC:2609-6,LOINC:2579-1,LOINC:26019-0,LOINC:33951-5,LOINC:26498-6,LOINC:2669-0,LOINC:30433-7,LOINC:42246-9,mode_height,median_weight,BMI,median_diastolic_value,median_systolic_value,ACS_MedHHIncome,ACS_GINI,ACS_Unemployment,ACS_pctPoverty100,ACS_pctCollGrad
i64,str,datetime[μs],i64,str,str,str,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f64,f32,f32,f32,f32,f32,f32,f32
576517,"""45-54""",null,0,"""F""","""05""","""N""","""W""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,65.0,208.0,34.609231,102.5,107.71154,76434.0,41.59,3.82166,3.36611,37.911701
5886210,"""55-64""",null,0,"""F""","""UNK""","""Y""","""W""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,…,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,63.0,255.5,45.254849,75.756668,91.0,76704.0,44.709999,6.7926,12.4702,30.646299
390161,"""18-34""",null,0,"""F""","""03""","""UNK""","""UNK""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,73399.0,46.189999,4.58634,15.5891,40.646801
5378498,"""55-64""",null,0,"""F""","""05""","""UNK""","""W""",0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,…,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,64.5,167.199997,28.253494,111.333336,129.570374,93160.0,39.09,5.84489,4.42763,40.234001
3476481,"""55-64""",null,0,"""F""","""UNK""","""UNK""","""UNK""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,59.5,127.5,25.318126,108.791664,108.791664,122599.0,46.139999,3.65209,12.5629,68.350899


In [115]:
np.unique(temp_df.select('Outcome').collect()['Outcome'].to_list(), return_counts = True)

(array([0, 1]), array([42123,  1501]))

In [116]:
np.unique(temp_df.select('Gender_CD').collect()['Gender_CD'].to_list(), return_counts = True)

(array(['GQ', 'M', 'TG', 'UNK', 'W'], dtype='<U3'),
 array([   94, 11477,   225,  9157, 22671]))

In [117]:
# Dicts to map 
age_group_dict = {'18-34': 0,
                  '35-44': 1,
                  '45-54':2,
                  '55-64':3,
                  '65-74':4,
                  '75_older':5    
}

# nominal encoding Gender, Sex and Race


In [118]:
temp_df = temp_df.with_columns(pl.col('Age_group').replace_strict(age_group_dict).alias('Age_group'))

In [119]:
temp_df.head().collect()

PATIENT_NUM,Age_group,FirstOutcomeDate,Outcome,Sex_CD,Race_CD,Hispanic_CD,Gender_CD,phenazopyridine,nitrofurantoin,lisinopril_hydrochlorothiazide,amlodipine,cephalexin,albuterol,nystatin,metronidazole,mometasone_formoterol,varicella-zoster virus vaccine live (Oka-Merck) strain,potassium chloride,chlorthalidone,ketorolac,azithromycin,triamcinolone,gabapentin,acetaminophen_hydrocodone,capsaicin,naproxen,tramadol,ibuprofen,codeine_promethazine,omeprazole,clotrimazole,lorazepam,fluticasone,calcium carbonate,loratadine,paroxetine,…,LOINC:41649-5,LOINC:13801-6,LOINC:49502-8,LOINC:15061-5,LOINC:6874-2,LOINC:32046-5,LOINC:2240-0,LOINC:3054-4,LOINC:6942-7,LOINC:29239-1,LOINC:42245-1,LOINC:1854-9,LOINC:35707-9,LOINC:2148-5,LOINC:38252-3,LOINC:72888-1,LOINC:83102-4,LOINC:38506-2,LOINC:26499-4,LOINC:2609-6,LOINC:2579-1,LOINC:26019-0,LOINC:33951-5,LOINC:26498-6,LOINC:2669-0,LOINC:30433-7,LOINC:42246-9,mode_height,median_weight,BMI,median_diastolic_value,median_systolic_value,ACS_MedHHIncome,ACS_GINI,ACS_Unemployment,ACS_pctPoverty100,ACS_pctCollGrad
i64,i64,datetime[μs],i64,str,str,str,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f64,f32,f32,f32,f32,f32,f32,f32
576517,2,null,0,"""F""","""05""","""N""","""W""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,65.0,208.0,34.609231,102.5,107.71154,76434.0,41.59,3.82166,3.36611,37.911701
5886210,3,null,0,"""F""","""UNK""","""Y""","""W""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,…,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,63.0,255.5,45.254849,75.756668,91.0,76704.0,44.709999,6.7926,12.4702,30.646299
390161,0,null,0,"""F""","""03""","""UNK""","""UNK""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,73399.0,46.189999,4.58634,15.5891,40.646801
5378498,3,null,0,"""F""","""05""","""UNK""","""W""",0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,…,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,64.5,167.199997,28.253494,111.333336,129.570374,93160.0,39.09,5.84489,4.42763,40.234001
3476481,3,null,0,"""F""","""UNK""","""UNK""","""UNK""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,59.5,127.5,25.318126,108.791664,108.791664,122599.0,46.139999,3.65209,12.5629,68.350899


In [120]:
if config.sdoh_screening_experiment:
    temp_df = temp_df.join(sdoh_screening_df, on = 'PATIENT_NUM', how = 'left')

In [121]:
temp_materialized_df = temp_df.collect()

# Define the columns to one-hot encode
columns_to_one_hot_encode = ['Sex_CD', 'Race_CD', 'Hispanic_CD', 'Gender_CD']

# Combine the non-encoded columns with the encoded ones
dt_modeling_cat_encoded_df = pl.concat([
    temp_materialized_df.select(pl.all().exclude(columns_to_one_hot_encode)),
    temp_materialized_df.select(columns_to_one_hot_encode).to_dummies()
], how='horizontal')

In [122]:
dt_modeling_cat_encoded_df.head()

PATIENT_NUM,Age_group,FirstOutcomeDate,Outcome,phenazopyridine,nitrofurantoin,lisinopril_hydrochlorothiazide,amlodipine,cephalexin,albuterol,nystatin,metronidazole,mometasone_formoterol,varicella-zoster virus vaccine live (Oka-Merck) strain,potassium chloride,chlorthalidone,ketorolac,azithromycin,triamcinolone,gabapentin,acetaminophen_hydrocodone,capsaicin,naproxen,tramadol,ibuprofen,codeine_promethazine,omeprazole,clotrimazole,lorazepam,fluticasone,calcium carbonate,loratadine,paroxetine,cholecalciferol,aluminum chloride,penicillin G,sulfamethoxazole_trimethoprim,…,SDH:ADV0104 _0.0,SDH:ADV0104 _1.0,SDH:ADV0261 _0.0,SDH:ADV0261 _1.0,SDH:ADV0114 _0.0,SDH:ADV0114 _1.0,SDH:ADV0115 _0.0,SDH:ADV0115 _1.0,SDH:ADV0116 _0.0,SDH:ADV0116 _1.0,SDH:ADV0117 _0.0,SDH:ADV0117 _1.0,SDH:ADV0118 _0.0,SDH:ADV0118 _1.0,SDH:ADV0262 _0.0,SDH:ADV0262 _1.0,SDH:ADV0400 _0.0,SDH:ADV0400 _1.0,SDH:ADV0401 _0.0,SDH:ADV0401 _1.0,Sex_CD_F,Sex_CD_M,Sex_CD_UNK,Race_CD_01,Race_CD_02,Race_CD_03,Race_CD_04,Race_CD_05,Race_CD_UNK,Hispanic_CD_N,Hispanic_CD_UNK,Hispanic_CD_Y,Gender_CD_GQ,Gender_CD_M,Gender_CD_TG,Gender_CD_UNK,Gender_CD_W
i64,i64,datetime[μs],i64,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8
576517,2,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1
5886210,3,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,…,0,0,0,0,0,1,1,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1
390161,0,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,1,1,0,0,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
5378498,3,null,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,…,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1
3476481,3,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0


In [123]:
# drop unk column in demogrpahics 'Sex_CD', 'Race_CD', 'Hispanic_CD', 'Gender_CD'
'Hispanic_CD_UNK' in dt_modeling_cat_encoded_df.columns

True

In [124]:
dt_modeling_cat_encoded_df = dt_modeling_cat_encoded_df.drop('Sex_CD_UNK','Race_CD_UNK','Hispanic_CD_UNK','Gender_CD_UNK')

In [125]:
[col for col in dt_modeling_cat_encoded_df.columns if col.startswith('Gender')]

['Gender_CD_GQ', 'Gender_CD_M', 'Gender_CD_TG', 'Gender_CD_W']

In [126]:
np.unique(dt_modeling_cat_encoded_df['Outcome'].to_list(), return_counts = True)

(array([0, 1]), array([42123,  1501]))

In [127]:
if 'initial' in config.diag_path:
    diag_prefix = 'initial_3_chars'
else:
    diag_prefix = 'phemap'

if 'without_icd10z' in config.diag_path:
    diag_prefix = diag_prefix+'_without_icd10z'
else:
    diag_prefix = diag_prefix +'_with_icd10z'
    
if "all_features" in config.lab_path:
    lab_prefix = 'all_features'
else:
    lab_prefix = 'domain_expert'

sdoh_prefix = ''    
if config.sdoh_screening_experiment:
    sdoh_prefix = '_SDOH_Screening'

if config.use_bmi_bp and config.use_cvs:
    dt_modeling_cat_encoded_df.write_parquet('./Determine_final_modeling_datasets/Determine_joined_med_'+'usage'
                      +'_lab_'+'median_'+lab_prefix
                      +'_diag_'+diag_prefix
                      +'_bmi_bp_'+ 'cvs'              
                      +'_ordinal_nominal_encoded'+sdoh_prefix+
                      '.parquet')

elif config.use_bmi_bp:
    dt_modeling_cat_encoded_df.write_parquet('./Determine_final_modeling_datasets/Determine_joined_med_'+'usage'
                      +'_lab_'+'median_'+lab_prefix
                      +'_diag_'+diag_prefix
                      +'_bmi_bp_'              
                      +'_ordinal_nominal_encoded'+sdoh_prefix+
                      '.parquet')


else:
    dt_modeling_cat_encoded_df.write_parquet('./Determine_final_modeling_datasets/Determine_joined_med_'+'usage'
                      +'_lab_'+'median_'+lab_prefix
                      +'_diag_'+diag_prefix
                      +'_ordinal_nominal_encoded'+sdoh_prefix+
                      '.parquet')

# Checking missing values for each patient

In [ ]:
modeling_df = pl.read_parquet('./Determine_final_modeling_datasets/Determine_joined_med_'+'usage'
                      +'_lab_'+'latest'
                      +'_diag_'+'phe_icd10'
                      +'_bmi_bp_'+ 'cvs'              
                      +'_ordinal_nominal_encoded'+
                      '.parquet')

In [ ]:
modeling_df = modeling_df.select([
    pl.col(col).cast(pl.Float32) if col.startswith('LOINC') else pl.col(col)
    for col in modeling_df.columns
])

In [ ]:
modeling_df.head()

In [ ]:
columns_to_ignore = [ 'Outcome', 'Age_group', 'FirstOutcomeDate'] + ['Sex_CD_F',
 'Sex_CD_M',
 #'Sex_CD_UNK',
 'Race_CD_01',
 'Race_CD_02',
 'Race_CD_03',
 'Race_CD_04',
 'Race_CD_05',
 #'Race_CD_06',
 #'Race_CD_07',
 #'Race_CD_UNK',
 'Hispanic_CD_N',
 #'Hispanic_CD_UNK',
 'Hispanic_CD_Y',
 'Gender_CD_GQ',
 'Gender_CD_M',
 'Gender_CD_TG',
 #'Gender_CD_UNK',
 'Gender_CD_W']

In [ ]:
modeling_df = modeling_df.drop(columns_to_ignore)

In [ ]:
lab_columns  = [col for col in modeling_df.columns if col.startswith('LOINC')]
med_diag_columns = list(set(modeling_df.columns) - set(lab_columns))

In [ ]:
print("Numbe of lab results columns", len(lab_columns))
print("Numbe of medications and diagnoses results columns", len(med_diag_columns))

In [ ]:
# sum_value_count_lab = []
# sum_value_count_med_diag = []
# for row in modeling_df.rows(named=True):
    
#     labs_dict = {key:row[key] for key in lab_columns}
#     med_diag_dict = {key:row[key] for key in med_diag_columns}
    
#     sum_value_count_lab.append(len(lab_columns) - list(labs_dict.values()).count(-100))
#     sum_value_count_med_diag.append(len(med_diag_columns) - list(med_diag_dict.values()).count(0))

In [ ]:
def count_values_in_med_diag(row:dict):
    return len(row) - list(row.values()).count(0)

def count_values_in_lab(row:dict):
    float_values = list(row.values())
    return len(float_values) - float_values.count(-100)

In [ ]:
counts_df = modeling_df.with_columns(
    pl.struct(pl.all().exclude(lab_columns)).map_elements(count_values_in_med_diag, return_dtype=pl.Int64).alias('non_null_med_diag_value_count'),
    pl.struct(pl.all().exclude(med_diag_columns)).map_elements(count_values_in_lab, return_dtype=pl.Int64).alias('non_null_lab_value_count')
    )

In [ ]:
counts_df.head()

In [ ]:
counts_df = counts_df.with_columns(pl.sum_horizontal(['non_null_med_diag_value_count','non_null_lab_value_count']).alias("Count_of_available_features"))

In [ ]:
counts_df.head()

In [ ]:
with open('catboost_false_positives_pids_test.pkl','rb') as f:
    fp_ids = pickle.load(f)

In [ ]:
import matplotlib.pyplot as plt
bins = 10 

data = counts_df['Count_of_available_features']

# Plotting the histogram
plt.xlim([min(data)-5, max(data)+5])
plt.hist(data, bins=bins, alpha=0.5)
# plt.xticks(range(10))
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Data points feature count')
plt.show()

## Check for key features missing rate in 